In [2]:
import os
import requests
from bs4 import BeautifulSoup
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#  1) Dataset preparation code




In [ ]:
google_image = "https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&"
user_agent = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
}

In [ ]:

def download_images(index,name):
    data = input('What are you looking for? ')
    n_images = int(input('How many images do you want? '))
    #class_id=input('Enter the class id: ')
    print('searching...')

    search_url = google_image + 'q=' + data

    response = requests.get(search_url, headers=user_agent)

    html = response.text
    html_=str(html)
    sarray=html_.split('img class=')[2]
    print(sarray)
    class_context=sarray[1:100]
    class_id=class_context.split('"')[0]
    print('class_id: ',class_id)
    soup = BeautifulSoup(html, 'html.parser')

    results = soup.findAll('img', {'class': str(class_id)})
    print('results: ', len(results))
    count = 1
    links = []
    for result in results:
        try:
            #print("result: ",result)
            link = result['src']
            #print("link: ",link)
            links.append(link)
            count += 1
            if(count > n_images):
                break

        except Exception as e:
            print(e.__class__)
            continue

    print(f"Downloading {len(links)} images...")
    for i, link in enumerate(links):
        response = requests.get(link)

        image_name = saved_folder + '/'+ name+'/'+ name+'_'+str(i+index) + '.jpg'

        with open(image_name, 'wb') as fh:
            fh.write(response.content)
        time.sleep(1)

In [ ]:

saved_folder = 'images'


def main():
    if not os.path.exists(saved_folder):
        os.mkdir(saved_folder)
        os.mkdir(saved_folder + '/peacock')
        os.mkdir(saved_folder + '/duck')
    for i in range(5):
        download_images(i*20,'peacock')
    for i in range(5):
        download_images(i*20,'duck')
    #     download_images(i*20,'duck')
main()

In [4]:
import torch
import torchvision
import torch.optim as optim
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, random_split,Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:

class ImageLabelDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data_dir = data_dir
    self.image_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
    self.transform = transform

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    image_path = self.image_paths[idx]
    # Load image as RGB (mode='RGB')
    image = Image.open(image_path).convert('RGB')
    # Assuming your labels are encoded in the filenames (modify as needed)
    label = os.path.splitext(os.path.basename(image_path))[0].split("_")[0]
    # Convert label to integer if needed (modify as needed)
    #print("LABEL: ",label)
    if label == 'peacock':
        label = [0,1]
    else:
        label = [1,0]
    if self.transform:
      image = self.transform(image)
    return image, label


In [6]:
data_dir = '/content/drive/My Drive/Assignment_4_data/images/combine'
data_transforms = transforms.Compose([
  transforms.Resize((200,200)),  # Resize images to 256x256
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.568, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageLabelDataset(data_dir,data_transforms)


In [7]:
print(dataset[0][0].shape)

torch.Size([3, 200, 200])


In [8]:
test_size = 0.2
train_size = int(len(dataset) * (1 - test_size))
train_dataset, test_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

# Create training and testing loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [9]:
import torch.nn as nn
class VGG(nn.Module):
    def __init__(
        self,
        architecture,
        in_channels=3,
        in_height=224,
        in_width=224,
        num_hidden=4096,
        num_classes=2
    ):
        super(VGG, self).__init__()
        self.in_channels = in_channels
        self.in_width = in_width
        self.in_height = in_height
        self.num_hidden = num_hidden
        self.num_classes = num_classes
        self.convs = self.init_convs(architecture)
        self.fcs = self.init_fcs(architecture)

    def forward(self, x):
        x = self.convs(x)
        x = x.reshape(x.size(0), -1)
        x = self.fcs(x)
        return x

    def init_fcs(self, architecture):
        pool_count = architecture.count("M")
        factor = (2 ** pool_count)
        if (self.in_height % factor) + (self.in_width % factor) != 0:
            raise ValueError(
                f"`in_height` and `in_width` must be multiples of {factor}"
            )
        out_height = self.in_height // factor
        out_width = self.in_width // factor
        last_out_channels = next(
            x for x in architecture[::-1] if type(x) == int
        )
        return nn.Sequential(
            nn.Linear(
                last_out_channels * out_height * out_width,
                self.num_hidden),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(self.num_hidden, self.num_hidden),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(self.num_hidden, self.num_classes)
            #nn.Softmax(dim=1)
        )


    def init_convs(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x
                layers.extend(
                    [
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3, 3),
                            stride=(1, 1),
                            padding=(1, 1),
                        ),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU(),
                    ]
                )
                in_channels = x
            else:
                layers.append(
                    nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
                )

        return nn.Sequential(*layers)


In [10]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64,64,"M",128,128,"M",256,256,"M",512,512,"M",512,512,"M",],
    "VGG16": [64,64,"M",128,128,"M",256,256,256,"M",512,512,512,"M",512,512,512,"M",],
    "VGG19": [64,64,"M",128,128,"M",256,256,256,256,"M",512,512,512,512,"M",512,512,512,512,"M",],
    "VGG1":[32,"M"],
    "VGG3": [32,"M",64,"M",128,"M"]
}

num_classes = 2
num_epochs = 20
total_step = len(train_loader)

In [11]:
import time
def train_moodel(model,train_loader,epochs,model_name):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    start_time = time.time()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to the configured device
            images = images.to(device)
            labels=torch.stack((labels[0], labels[1]),dim=1)
            labels=labels.float()
            #print("labels: ",labels)
            labels = labels.to(device)
            #print("shape of labels: ",labels.shape)
            # Forward pass
            outputs = model(images)
            #print("shape of outputs: ",outputs.shape)
            #print("outputs: ",outputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
            #final_loss=final_loss+loss.item()
            #print("loss: ",loss.item())
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            del images,labels
        train_loss = running_loss / len(train_loader.dataset)
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, train_loss))
    end_time = time.time()
    torch.save(model.state_dict(),f"/content/drive/My Drive/Assignment_4_data/{model_name}.pt")
    print('Time: ',end_time - start_time)
    return (end_time - start_time),train_loss

In [12]:
def find_train_accuracy(model,train_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in train_loader:
            images = images.to(device)
            labels=torch.stack((labels[0], labels[1]),dim=1)
            labels=labels.float()
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total) * 100

def find_test_accuracy(model,test_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels=torch.stack((labels[0], labels[1]),dim=1)
            labels=labels.float()
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return (correct / total) * 100

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [13]:
model_details={"Model_Name":[],"Traning_Time":[],"Training_Loss":[],"Train_Accuracy":[],"Test_Accuracy":[],"Number_Of_Parameters":[]}

# I) Training VGG 1 block model

In [ ]:
model = VGG(VGG_types["VGG1"], in_channels=3, in_height=200, in_width=200, num_hidden=1024, num_classes=2)
train_time,loss=train_moodel(model,train_loader,100,"vgg1")
train_accuracy=find_train_accuracy(model,train_loader)
test_accuracy=find_test_accuracy(model,test_loader)
total_params=count_parameters(model)
model_details["Model_Name"].append("VGG 1 Block")
model_details["Traning_Time"].append(train_time)
model_details["Train_Accuracy"].append(train_accuracy)
model_details["Test_Accuracy"].append(test_accuracy)
model_details["Number_Of_Parameters"].append(total_params)
model_details["Training_Loss"].append(loss)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print("Train Time: ",train_time)
print("Loss: ",loss)
print("total_params: ",total_params)

Epoch [1/100], Loss: 38.1904
Epoch [2/100], Loss: 20.1055
Epoch [3/100], Loss: 10.0322
Epoch [4/100], Loss: 7.4360
Epoch [5/100], Loss: 4.2223
Epoch [6/100], Loss: 2.7058
Epoch [7/100], Loss: 1.2219
Epoch [8/100], Loss: 1.1971
Epoch [9/100], Loss: 1.0267
Epoch [10/100], Loss: 1.2297
Epoch [11/100], Loss: 1.4437
Epoch [12/100], Loss: 0.9336
Epoch [13/100], Loss: 0.3458
Epoch [14/100], Loss: 0.1755
Epoch [15/100], Loss: 0.5947
Epoch [16/100], Loss: 0.3628
Epoch [17/100], Loss: 0.2865
Epoch [18/100], Loss: 0.3806
Epoch [19/100], Loss: 0.2772
Epoch [20/100], Loss: 0.3133
Epoch [21/100], Loss: 0.1175
Epoch [22/100], Loss: 0.0435
Epoch [23/100], Loss: 0.1487
Epoch [24/100], Loss: 0.0298
Epoch [25/100], Loss: 0.0342
Epoch [26/100], Loss: 0.0129
Epoch [27/100], Loss: 0.0842
Epoch [28/100], Loss: 0.0664
Epoch [29/100], Loss: 0.0367
Epoch [30/100], Loss: 0.0236
Epoch [31/100], Loss: 0.1904
Epoch [32/100], Loss: 0.0317
Epoch [33/100], Loss: 0.0698
Epoch [34/100], Loss: 0.0552
Epoch [35/100], Loss

# II) Training VGG 3 block model

In [ ]:
model = VGG(VGG_types["VGG3"], in_channels=3, in_height=200, in_width=200, num_hidden=1024, num_classes=2)
train_time,loss=train_moodel(model,train_loader,100,"vgg3")
train_accuracy=find_train_accuracy(model,train_loader)
test_accuracy=find_test_accuracy(model,test_loader)
total_params=count_parameters(model)
model_details["Model_Name"].append("VGG 3 Block")
model_details["Traning_Time"].append(train_time)
model_details["Train_Accuracy"].append(train_accuracy)
model_details["Test_Accuracy"].append(test_accuracy)
model_details["Number_Of_Parameters"].append(total_params)
model_details["Training_Loss"].append(loss)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print("Train Time: ",train_time)
print("Loss: ",loss)
print("total_params: ",total_params)

Epoch [1/100], Loss: 14.2095
Epoch [2/100], Loss: 6.1025
Epoch [3/100], Loss: 3.6355
Epoch [4/100], Loss: 1.8387
Epoch [5/100], Loss: 1.3341
Epoch [6/100], Loss: 0.9244
Epoch [7/100], Loss: 0.5930
Epoch [8/100], Loss: 0.5118
Epoch [9/100], Loss: 0.3865
Epoch [10/100], Loss: 0.4103
Epoch [11/100], Loss: 0.2682
Epoch [12/100], Loss: 0.2375
Epoch [13/100], Loss: 0.1596
Epoch [14/100], Loss: 0.1448
Epoch [15/100], Loss: 0.1347
Epoch [16/100], Loss: 0.1867
Epoch [17/100], Loss: 0.1017
Epoch [18/100], Loss: 0.0784
Epoch [19/100], Loss: 0.0795
Epoch [20/100], Loss: 0.0822
Epoch [21/100], Loss: 0.0925
Epoch [22/100], Loss: 0.0247
Epoch [23/100], Loss: 0.0364
Epoch [24/100], Loss: 0.0338
Epoch [25/100], Loss: 0.0735
Epoch [26/100], Loss: 0.0722
Epoch [27/100], Loss: 0.0468
Epoch [28/100], Loss: 0.0734
Epoch [29/100], Loss: 0.1226
Epoch [30/100], Loss: 0.0521
Epoch [31/100], Loss: 0.0556
Epoch [32/100], Loss: 0.0516
Epoch [33/100], Loss: 0.0549
Epoch [34/100], Loss: 0.0589
Epoch [35/100], Loss: 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        print(labels)
        labels = labels[1].to(device)
        outputs = model(images)
        print(outputs)
        logits, predicted = torch.max(outputs.data, 1)
        print(logits)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

[tensor([0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 1, 0]), tensor([1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 1])]
tensor([[ -9.4400,   9.3858],
        [ 35.0892, -37.8104],
        [ -3.8872,   6.7910],
        [ -3.3864,   4.7584],
        [ -7.7785,   6.8084],
        [  2.2754,  -4.5363],
        [ -7.0080,   7.1373],
        [  5.6260,  -6.0164],
        [  2.4081,  -1.8405],
        [ -3.0430,  11.5041],
        [ 43.4163, -33.8768],
        [-14.7671,  24.9637],
        [  3.7345,  -5.0211],
        [ -6.1299,   9.6998],
        [ -1.8023,   2.1509],
        [ -0.1125,  -0.8359],
        [ -4.1570,   5.0758],
        [ -0.5833,   2.3651],
        [ -0.5744,   1.7246],
        [  1.4415,  -3.5096],
        [ -0.2319,   1.1401],
        [-14.9475,  26.2135],
        [-13.7639,  17.8193],
        [ -5.0125,   5.6471],
        [  5.2643,  -5.8663],
        [  0.9985,   

In [ ]:
data_transforms_ = transforms.Compose([

  transforms.ToTensor(),
  transforms.RandomHorizontalFlip(p=1),
  transforms.RandomVerticalFlip(p=1),
  transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
  transforms.RandomRotation(degrees=(-90,90))
])

In [ ]:
# from torchvision import datasets
src_dir="/content/drive/My Drive/Assignment_4_data/images/combine/"
dest_dir="/content/drive/My Drive/Assignment_4_data/images/augmented_images/"
# dataset = datasets.ImageFolder(root=src_dir)
# print(dataset)
if not os.path.exists(dest_dir):
        os.mkdir(dest_dir)
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(src_dir) if isfile(join(src_dir, f))]
for file in onlyfiles:
  image = Image.open(src_dir+file).convert('RGB')
  image_=data_transforms_(image)
  image_ = transforms.ToPILImage()(image_)
  label = os.path.splitext(file)[0].split("_")
  image_.save(dest_dir+f"{label[0]}_a_{label[1]}.jpg")
  image.save(dest_dir+f"{label[0]}_{label[1]}.jpg")

In [ ]:
data_transforms = transforms.Compose([
  transforms.Resize((200,200)),  # Resize images to 256x256
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.485, 0.568, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset_ = ImageLabelDataset(dest_dir,data_transforms)

test_size = 0.2
train_size = int(len(dataset) * (1 - test_size))
train_dataset_, test_dataset_ = random_split(dataset_, [train_size, len(dataset_) - train_size])

# Create training and testing loaders
train_loader_ = torch.utils.data.DataLoader(train_dataset_, batch_size=32, shuffle=True)
test_loader_ = torch.utils.data.DataLoader(test_dataset_, batch_size=32, shuffle=False)

 # III) Training VGG 3 block model with data augmentation

In [ ]:
model = VGG(VGG_types["VGG3"], in_channels=3, in_height=200, in_width=200, num_hidden=1024, num_classes=2)
train_time,loss=train_moodel(model,train_loader_,100,"vgg3_augmented")
train_accuracy=find_train_accuracy(model,train_loader_)
test_accuracy=find_test_accuracy(model,test_loader_)
total_params=count_parameters(model)
model_details["Model_Name"].append("VGG 3 Block Augmented")
model_details["Traning_Time"].append(train_time)
model_details["Train_Accuracy"].append(train_accuracy)
model_details["Test_Accuracy"].append(test_accuracy)
model_details["Number_Of_Parameters"].append(total_params)
model_details["Training_Loss"].append(loss)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print("Train Time: ",train_time)
print("Loss: ",loss)
print("total_params: ",total_params)

Epoch [1/100], Loss: 10.0537
Epoch [2/100], Loss: 2.2629
Epoch [3/100], Loss: 1.3283
Epoch [4/100], Loss: 0.8679
Epoch [5/100], Loss: 0.8179
Epoch [6/100], Loss: 0.6776
Epoch [7/100], Loss: 0.5886
Epoch [8/100], Loss: 0.5861
Epoch [9/100], Loss: 0.6191
Epoch [10/100], Loss: 0.5719
Epoch [11/100], Loss: 0.5445
Epoch [12/100], Loss: 0.4946
Epoch [13/100], Loss: 0.5421
Epoch [14/100], Loss: 0.4754
Epoch [15/100], Loss: 0.3786
Epoch [16/100], Loss: 0.4014
Epoch [17/100], Loss: 0.3776
Epoch [18/100], Loss: 0.4736
Epoch [19/100], Loss: 0.4181
Epoch [20/100], Loss: 0.3347
Epoch [21/100], Loss: 0.3008
Epoch [22/100], Loss: 0.3599
Epoch [23/100], Loss: 0.3624
Epoch [24/100], Loss: 0.2817
Epoch [25/100], Loss: 0.2835
Epoch [26/100], Loss: 0.3113
Epoch [27/100], Loss: 0.2604
Epoch [28/100], Loss: 0.2424
Epoch [29/100], Loss: 0.2843
Epoch [30/100], Loss: 0.3202
Epoch [31/100], Loss: 0.2639
Epoch [32/100], Loss: 0.2670
Epoch [33/100], Loss: 0.2309
Epoch [34/100], Loss: 0.2472
Epoch [35/100], Loss: 

In [ ]:
import pandas as pd

df=pd.DataFrame(model_details,index=None)
print(df)
df.to_csv("/content/drive/My Drive/Assignment_4_data/model_details.csv")

              Model_Name  Traning_Time  Training_Loss  Train_Accuracy  \
0            VGG 1 Block     84.426020       0.062660           100.0   
1            VGG 3 Block     80.806856       0.001187           100.0   
2  VGG 3 Block Augmented    164.781590       0.101514           100.0   

   Test_Accuracy  Number_Of_Parameters  
0           75.0             328733634  
1           80.0              83066370  
2           67.5              83066370  


# IV) Training VGG 16 block model with transfer learning by tuning all layers(including convolution layers)

In [39]:
import torchvision.models as models
model = models.vgg16(weights='DEFAULT')
in_height = 224
in_width = 224

pool_count = VGG_types["VGG16"].count("M")
factor = (2 ** pool_count)
if (in_height % factor) + (in_width % factor) != 0:
    raise ValueError(
                f"`in_height` and `in_width` must be multiples of {factor}"
    )
out_height = in_height // factor
out_width = in_width // factor
last_out_channels = next(
            x for x in VGG_types["VGG16"][::-1] if type(x) == int
    )
model.classifier=nn.Sequential(nn.Linear(last_out_channels*out_height*out_width,1024),nn.ReLU(),nn.Dropout(p=0.5), nn.Linear(1024,1024),nn.ReLU(),nn.Dropout(p=0.5), nn.Linear(1024,2))
train_time,loss=train_moodel(model,train_loader,100,"vgg16_Transfer_Learning_1")
train_accuracy=find_train_accuracy(model,train_loader)
test_accuracy=find_test_accuracy(model,test_loader)
total_params=count_parameters(model)
model_details["Model_Name"].append("VGG 16 Transfer Learning 1")
model_details["Traning_Time"].append(train_time)
model_details["Train_Accuracy"].append(train_accuracy)
model_details["Test_Accuracy"].append(test_accuracy)
model_details["Number_Of_Parameters"].append(total_params)
model_details["Training_Loss"].append(loss)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print("Train Time: ",train_time)
print("Loss: ",loss)
print("total_params: ",total_params)

Epoch [1/100], Loss: 0.7167
Epoch [2/100], Loss: 1.5960
Epoch [3/100], Loss: 0.8207
Epoch [4/100], Loss: 0.6927
Epoch [5/100], Loss: 0.6916
Epoch [6/100], Loss: 0.6920
Epoch [7/100], Loss: 0.6902
Epoch [8/100], Loss: 0.6919
Epoch [9/100], Loss: 0.6898
Epoch [10/100], Loss: 0.6732
Epoch [11/100], Loss: 0.6778
Epoch [12/100], Loss: 0.6046
Epoch [13/100], Loss: 0.5430
Epoch [14/100], Loss: 0.5788
Epoch [15/100], Loss: 0.5551
Epoch [16/100], Loss: 0.5031
Epoch [17/100], Loss: 0.3896
Epoch [18/100], Loss: 0.2639
Epoch [19/100], Loss: 0.3165
Epoch [20/100], Loss: 0.2534
Epoch [21/100], Loss: 0.1886
Epoch [22/100], Loss: 1.4858
Epoch [23/100], Loss: 0.8517
Epoch [24/100], Loss: 0.6952
Epoch [25/100], Loss: 0.6953
Epoch [26/100], Loss: 0.6940
Epoch [27/100], Loss: 0.6939
Epoch [28/100], Loss: 0.6934
Epoch [29/100], Loss: 0.6934
Epoch [30/100], Loss: 0.6930
Epoch [31/100], Loss: 0.6928
Epoch [32/100], Loss: 0.6922
Epoch [33/100], Loss: 0.6923
Epoch [34/100], Loss: 0.6921
Epoch [35/100], Loss: 0

# V) Training VGG 16 block model with transfer learning by tuning only MLP layers

In [15]:
import torchvision.models as models
model = models.vgg16(weights='DEFAULT')
in_height = 224
in_width = 224

pool_count = VGG_types["VGG16"].count("M")
factor = (2 ** pool_count)
if (in_height % factor) + (in_width % factor) != 0:
    raise ValueError(
                f"`in_height` and `in_width` must be multiples of {factor}"
    )
out_height = in_height // factor
out_width = in_width // factor
last_out_channels = next(
            x for x in VGG_types["VGG16"][::-1] if type(x) == int
    )
for param in model.features.parameters():
    param.requires_grad = False
model.classifier=nn.Sequential(nn.Linear(last_out_channels*out_height*out_width,1024),nn.ReLU(),nn.Dropout(p=0.5), nn.Linear(1024,1024),nn.ReLU(),nn.Dropout(p=0.5), nn.Linear(1024,2))
train_time,loss=train_moodel(model,train_loader,100,"vgg16_Transfer_Learning_1")
train_accuracy=find_train_accuracy(model,train_loader)
test_accuracy=find_test_accuracy(model,test_loader)
total_params=count_parameters(model)
model_details["Model_Name"].append("VGG 16 Transfer Learning 1")
model_details["Traning_Time"].append(train_time)
model_details["Train_Accuracy"].append(train_accuracy)
model_details["Test_Accuracy"].append(test_accuracy)
model_details["Number_Of_Parameters"].append(total_params)
model_details["Training_Loss"].append(loss)
print("Train Accuracy: ",train_accuracy)
print("Test Accuracy: ",test_accuracy)
print("Train Time: ",train_time)
print("Loss: ",loss)
print("total_params: ",total_params)

Epoch [1/100], Loss: 1.2495
Epoch [2/100], Loss: 0.3315
Epoch [3/100], Loss: 0.1070
Epoch [4/100], Loss: 0.0274
Epoch [5/100], Loss: 0.0171
Epoch [6/100], Loss: 0.0025
Epoch [7/100], Loss: 0.0006
Epoch [8/100], Loss: 0.0006
Epoch [9/100], Loss: 0.0001
Epoch [10/100], Loss: 0.0002
Epoch [11/100], Loss: 0.0000
Epoch [12/100], Loss: 0.0002
Epoch [13/100], Loss: 0.0004
Epoch [14/100], Loss: 0.0000
Epoch [15/100], Loss: 0.0000
Epoch [16/100], Loss: 0.0000
Epoch [17/100], Loss: 0.0000
Epoch [18/100], Loss: 0.0000
Epoch [19/100], Loss: 0.0001
Epoch [20/100], Loss: 0.0000
Epoch [21/100], Loss: 0.0000
Epoch [22/100], Loss: 0.0000
Epoch [23/100], Loss: 0.0000
Epoch [24/100], Loss: 0.0000
Epoch [25/100], Loss: 0.0000
Epoch [26/100], Loss: 0.0000
Epoch [27/100], Loss: 0.0000
Epoch [28/100], Loss: 0.0000
Epoch [29/100], Loss: 0.0000
Epoch [30/100], Loss: 0.0000
Epoch [31/100], Loss: 0.0000
Epoch [32/100], Loss: 0.0000
Epoch [33/100], Loss: 0.0000
Epoch [34/100], Loss: 0.0000
Epoch [35/100], Loss: 0

In [25]:
df=pd.read_csv("/content/drive/My Drive/Assignment_4_data/model_details.csv",index_col=False)
df = df.drop(df.columns[0], axis=1)
df.loc[len(df.index)]=["VGG 16 Transfer Learning 1",177.98576045036316,0.6908,53.125,37.5,41457474]
df.loc[len(df.index)]=["VGG 16 Transfer Learning 2",134.33612298965454,0.0000,100.0,87.5,26742786]

df.to_csv("/content/drive/My Drive/Assignment_4_data/model_details.csv",index=False)
df

,Model_Name,Traning_Time,Training_Loss,Train_Accuracy,Test_Accuracy,Number_Of_Parameters
0,VGG 1 Block,84.426020,0.062660,100.000,75.0,328733634
1,VGG 3 Block,80.806856,0.001187,100.000,80.0,83066370
2,VGG 3 Block Augmented,164.781590,0.101514,100.000,67.5,83066370
3,VGG 16 Transfer Learning 1,177.985760,0.690800,53.125,37.5,41457474
4,VGG 16 Transfer Learning 2,134.336123,0.000000,100.000,87.5,26742786
